In [194]:
import pandas as pd
from datetime import timedelta

filepath1 = r'/Users/cartersocha/Downloads/uberDataset6.csv'
uberSet = pd.read_csv(filepath1)

In [195]:
uberSet2 = uberSet
uberSet2 = uberSet2.groupby(['TvShow','PostReleaseDay']).sum().reset_index()
uberSet2['moving'] = uberSet2.groupby(['TvShow','PostReleaseDay'])['ScaledDataAdjust-QuantileTransformer'].transform(lambda x: x.rolling(7,1).mean())

### Moving average

In [196]:
uberSet2.head(15)

,TvShow,PostReleaseDay,Unnamed: 0,SocialMediaValue,ScaledDataAdjust-QuantileTransformer,ScaledDataAdjust-MinMax,ScaledDataAdjust-StandardScaler,ScaledDataAdjust-MaxAbsScaler,ScaledDataAdjust-RobustScaler,ScaledDataAdjust-Normalizer,ScaledDataAdjust-PowerTransformer,EpisodeCount,SeasonNumber,Ignore,DailyReleaseCount,moving
0,ABlackLadySketchShow,0,10745,360.64,1.993450,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,5.0,1.993450
1,ABlackLadySketchShow,1,8614,1077.00,2.639429,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,2.639429
2,ABlackLadySketchShow,2,10790,1801.24,2.677893,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,2.677893
3,ABlackLadySketchShow,3,10815,834.76,2.339414,0.000082,-0.207856,0.000038,-0.225225,1.0,-1.274430,30.0,15.0,5.0,0.0,2.339414
4,ABlackLadySketchShow,4,10840,458.64,2.075665,0.000082,-0.207856,0.000038,-0.225225,1.0,-1.274430,30.0,15.0,5.0,0.0,2.075665
5,ABlackLadySketchShow,5,10865,278.16,1.884641,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,1.884641
6,ABlackLadySketchShow,6,8710,239.44,1.216671,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,1.216671
7,ABlackLadySketchShow,7,10910,180.25,1.649437,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,5.0,1.649437
8,ABlackLadySketchShow,8,8746,355.24,1.808895,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,24.0,12.0,4.0,0.0,1.808895
9,ABlackLadySketchShow,9,10955,919.76,2.280650,0.000057,-0.208537,0.000013,-0.234234,1.0,-1.660078,30.0,15.0,5.0,0.0,2.280650


In [197]:
firstWeek = uberSet[uberSet['PostReleaseDay'] < 7]
firstWeek = firstWeek[firstWeek['Ignore'] == 1]
firstWeek['ScaledDataAdjust-QuantileTransformer'] = firstWeek['ScaledDataAdjust-QuantileTransformer'].replace(0,.1)
firstWeek['SocialMediaValue'] = firstWeek['SocialMediaValue'].replace(0,.1)
firstWeekSum = firstWeek.groupby(['TvShow','ReleaseCadence'], as_index=False).sum()
firstWeekSum = firstWeekSum[['TvShow','ReleaseCadence','SocialMediaValue','ScaledDataAdjust-QuantileTransformer']]

In [198]:
totalDf = uberSet[uberSet['Ignore'] == 1]
totalDf['SocialMediaValue'] = totalDf['SocialMediaValue'].replace(0,.1)
totalDf = totalDf.groupby(['TvShow','ReleaseCadence'], as_index=False).sum()
totalDf = totalDf[['TvShow','ReleaseCadence','SocialMediaValue','ScaledDataAdjust-QuantileTransformer']]

In [199]:
firstWeekSum.head()

,TvShow,ReleaseCadence,SocialMediaValue,ScaledDataAdjust-QuantileTransformer
0,ABlackLadySketchShow,Weekly,5049.88,14.827163
1,AboutLastNighthbo,Binge,0.70,0.700000
2,Archive81,Binge,42649.28,16.023360
3,Atlantafx,Hybrid,402624.73,26.995722
4,Barry,Weekly,212344.21,28.224639


In [200]:
totalDf.sort_values(['ScaledDataAdjust-QuantileTransformer']).head()

,TvShow,ReleaseCadence,SocialMediaValue,ScaledDataAdjust-QuantileTransformer
1,AboutLastNighthbo,Binge,9.80,0.148148
53,tehrantv,Hybrid,8116.87,59.833905
51,WokeHulu,Binge,6517.23,69.596446
15,HypeHouse,Binge,66522.44,73.989323
17,JoeVsCarole,Binge,10007.52,74.782577


In [201]:
merged = pd.merge(totalDf,firstWeekSum, how='outer', on='TvShow')

In [202]:
merged['multiplier'] = merged['ScaledDataAdjust-QuantileTransformer_x'] / merged['ScaledDataAdjust-QuantileTransformer_y']

In [203]:
merged['multiplier2'] = merged['SocialMediaValue_x'] / merged['SocialMediaValue_y']

In [204]:
merged[['TvShow','ReleaseCadence_x','multiplier','multiplier2']].sort_values(['multiplier']).head()

,TvShow,ReleaseCadence_x,multiplier,multiplier2
1,AboutLastNighthbo,Binge,0.211640,14.000000
51,WokeHulu,Binge,5.486796,2.884828
17,JoeVsCarole,Binge,5.499899,2.051041
6,BlingEmpire,Binge,5.801197,1.793964
8,ConversationsWithFriends,Binge,5.846969,2.083962


In [205]:
analysisDf = merged.groupby(['ReleaseCadence_x'], as_index=False).mean()

In [206]:
analysisDf

,ReleaseCadence_x,SocialMediaValue_x,ScaledDataAdjust-QuantileTransformer_x,SocialMediaValue_y,ScaledDataAdjust-QuantileTransformer_y,multiplier,multiplier2
0,Binge,7.609294e+05,192.066686,209877.012105,22.325345,8.172213,5.136646
1,Hybrid,2.052261e+06,243.994278,348437.618400,23.162813,10.778315,7.047054
2,Weekly,2.074992e+06,259.795698,395781.323636,23.692448,11.897911,7.415008
